# Preprocess Eng

In [1]:
# import library
import re
import string
import progressbar
import numpy as np
import pandas as pd
from tqdm import tqdm 
import ipython_genutils
from sklearn import utils
from bs4 import BeautifulSoup
from keras.layers import Flatten
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from gensim.models.doc2vec import TaggedDocument

Using TensorFlow backend.


### load file 

In [2]:
#load file
# file 1
dfct = pd.read_csv('data/clean_tweet.csv',index_col=0)
dfct['target']= dfct['target'].map({0:0,4:2})
dfct.dropna(inplace=True)
dfct.reset_index(drop=True,inplace=True)
dfct.info()
print()

#file 2
airlinedf = pd.read_csv('data/airline.csv',encoding='ISO-8859-1')
airlinedf['airline_sentiment'] = airlinedf['airline_sentiment'].map({'negative':0,'neutral':1,'positive':2})
airlinedf.dropna()
airlinedf.reset_index(drop=True,inplace=True)
airlinedf.info()
print()

#file 3
debatedf = pd.read_csv('data/debate.csv',encoding='ISO-8859-1')
debatedf['sentiment']=debatedf['sentiment'].map({'Negative':0,'Neutral':1,'Positive':2})
debatedf.info()

C:\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596753 entries, 0 to 1596752
Data columns (total 2 columns):
text      1596753 non-null object
target    1596753 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.4+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 20 columns):
_unit_id                        14640 non-null int64
_golden                         14640 non-null bool
_unit_state                     14640 non-null object
_trusted_judgments              14640 non-null int64
_last_judgment_at               14584 non-null object
airline_sentiment               14640 non-null int64
airline_sentiment:confidence    14640 non-null float64
negativereason                  9178 non-null object
negativereason:confidence       10522 non-null float64
airline                         14640 non-null object
airline_sentiment_gold          40 non-null object
name                            14640 non-null object
negativereason_gold

In [3]:
# gabung data dari file
textdf=dfct['text'].astype('str')
textair=airlinedf['text'].astype('str')
textdebate=debatedf['text'].astype('str')
x=pd.concat([textdf,textair,textdebate])
x.reset_index(drop=True,inplace=True)

targetdf=dfct['target']
targetair=airlinedf['airline_sentiment']
targetdebate=debatedf['sentiment']
y=pd.concat([targetdf,targetair,targetdebate])
y.reset_index(drop=True,inplace=True)

df=pd.DataFrame({'text':x,'target':y})
df.reset_index(drop=True,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625264 entries, 0 to 1625263
Data columns (total 2 columns):
text      1625264 non-null object
target    1625264 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.8+ MB


### hapus emoji

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
widgets = [progressbar.Percentage(),progressbar.Bar()," Processed : ",progressbar.Counter(),"  ",progressbar.ETA()]
bar = progressbar.ProgressBar(widgets=widgets, max_value=len(df.index))
bar.start()

t=[]
for index,row in bar(df.iterrows()):
    t.append(remove_emoji(row['text']))
df.text=t

### cleaning

In [4]:
# tweet cleaning
tok=WordPunctTokenizer()
t=[]
for index,row in df.iterrows():
    x=row["text"]
    # hapus rt
    cl = re.sub(r'\s*RT\s*@[^:]*:.*', '', x)
    cl = re.sub(r'\s*rt\s*@[^:]*:.*', '', cl)
    # hapus mention
    cl = re.sub(r'@[A-Za-z0-9]([^:\s]+)+', '', cl)
    # hapus link
    cl = re.sub(r'https?://[A-Za-z0-9./]+', '', cl)
    # hapus hashtag
    cl = re.sub(r'(?:\s|^)#[A-Za-z0-9\-\.\_]+(?:\s|$)', '', cl)
    # kata ulang
    cl = re.sub(r'\w*\d\w*', '', cl)
    cl = re.sub(r'\b(\w+)(\1\b)+', r'\1', cl)
    # hapus simbol
    cl = re.sub(r'[^a-zA-Z]', ' ', cl)
    # lower
    cl=cl.lower()
    # format teks 
    cl=tok.tokenize(cl)
    cl=(" ".join(cl))
    t.append(cl)
    #print(cl)
df.text=t
df.head()

,text,target
0,awww that s a bummer you shoulda got david car...,0
1,is upset that he can t update his facebook by ...,0
2,i dived many times for the ball managed to sav...,0
3,my whole body feels itchy and like its on fire,0
4,no it s not behaving at all i m mad why am i h...,0


### hapus row yg terdapat data kosong

In [5]:
# hapus data dgn missing value
ex=''
df=df[df.text != ex]
df.reset_index(drop=True,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1618059 entries, 0 to 1618058
Data columns (total 2 columns):
text      1618059 non-null object
target    1618059 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.7+ MB


### hapus stopwords

In [9]:
# stopwords
stopword_ = set(stopwords.words('english'))
t=[]
c=0
for index,row in df.iterrows():
    tokens = word_tokenize(row['text'])
    result = [i for i in tokens if not i in stopword_]
    result=' '.join(result)
    t.append(result)
    if c%100000==0:
        print('data processed : ',c)
    c+=1
df.text=t
df.head()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

,text,target
0,awww bummer shoulda got david carr third day,0
1,upset update facebook texting might cry result...,0
2,dived many times ball managed save rest go bounds,0
3,whole body feels itchy like fire,0
4,behaving mad see,0


### stemming

In [10]:
# stemming
stemmer= PorterStemmer()
t=[]
for index,row in df.iterrows():
    result=[]
    text = word_tokenize(row['text'])
    for word in text:
        result.append(stemmer.stem(word))
    result=' '.join(result) 
    t.append(result)
df.text=t

data processed :  0
data processed :  100000
data processed :  200000
data processed :  300000
data processed :  400000
data processed :  500000
data processed :  600000
data processed :  700000
data processed :  800000
data processed :  900000
data processed :  1000000
data processed :  1100000
data processed :  1200000
data processed :  1300000
data processed :  1400000
data processed :  1500000
data processed :  1600000


,text,target
0,awww bummer shoulda got david carr third day,0
1,upset updat facebook text might cri result sch...,0
2,dive mani time ball manag save rest go bound,0
3,whole bodi feel itchi like fire,0
4,behav mad see,0


### save file

In [11]:
# save ke csv
df.to_csv('clean_text.csv')